In [1]:
import pandas as pd
import folium as fl
from folium.plugins import HeatMap, HeatMapWithTime

In [3]:
dfTrain = pd.read_csv(r"D:\Open Classroom\Datasets\nyc-taxi-trip-duration\train\train.csv").drop(columns = ["trip_duration", "dropoff_datetime"])
dfTest = pd.read_csv(r"D:\Open Classroom\Datasets\nyc-taxi-trip-duration\test\test.csv")
df = pd.concat([dfTrain, dfTest], sort = False, ignore_index = True)

In [4]:
df.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id2875421,2,2016-03-14 17:24:55,1,-73.982155,40.767937,-73.964630,40.765602,N
1,id2377394,1,2016-06-12 00:43:35,1,-73.980415,40.738564,-73.999481,40.731152,N
2,id3858529,2,2016-01-19 11:35:24,1,-73.979027,40.763939,-74.005333,40.710087,N
3,id3504673,2,2016-04-06 19:32:31,1,-74.010040,40.719971,-74.012268,40.706718,N
4,id2181028,2,2016-03-26 13:30:55,1,-73.973053,40.793209,-73.972923,40.782520,N


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2083778 entries, 0 to 2083777
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   id                  object 
 1   vendor_id           int64  
 2   pickup_datetime     object 
 3   passenger_count     int64  
 4   pickup_longitude    float64
 5   pickup_latitude     float64
 6   dropoff_longitude   float64
 7   dropoff_latitude    float64
 8   store_and_fwd_flag  object 
dtypes: float64(4), int64(2), object(3)
memory usage: 143.1+ MB


In [6]:
df.shape

(2083778, 9)

In [7]:
df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"], format = "%Y-%m-%d %H:%M:%S")
df["month"] = df["pickup_datetime"].apply(lambda x: x.month)
df["week"] = df["pickup_datetime"].apply(lambda x: x.week)
df["day"] = df["pickup_datetime"].apply(lambda x: x.day)
df["hours"] = df["pickup_datetime"].apply(lambda x: x.hour)

In [8]:
df.head()
#df.info()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,month,week,day,hours
0,id2875421,2,2016-03-14 17:24:55,1,-73.982155,40.767937,-73.964630,40.765602,N,3,11,14,17
1,id2377394,1,2016-06-12 00:43:35,1,-73.980415,40.738564,-73.999481,40.731152,N,6,23,12,0
2,id3858529,2,2016-01-19 11:35:24,1,-73.979027,40.763939,-74.005333,40.710087,N,1,3,19,11
3,id3504673,2,2016-04-06 19:32:31,1,-74.010040,40.719971,-74.012268,40.706718,N,4,14,6,19
4,id2181028,2,2016-03-26 13:30:55,1,-73.973053,40.793209,-73.972923,40.782520,N,3,12,26,13


In [9]:
def generateBaseMap(loc = [40.693943, -73.985880], zoom = 12):
    baseMap = fl.Map(location = loc, control_scale = True, zoom_start = zoom)
    fl.Marker([40.693943, -73.985880], tooltip = "<b> Click for more info </b>", icon = fl.Icon(icon = "train", prefix = "fa", color = "red")).add_to(baseMap)
    return baseMap

In [10]:
baseMap = generateBaseMap()
baseMap

In [11]:
dfCopy = df["month"] > 4
dfCopy["count"] = 1
baseMap = generateBaseMap()

In [13]:
new_df = pd.read_csv(r"D:\Open Classroom\Datasets\nyc-taxi-trip-duration\train\train.csv")
new_df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [19]:

new_df["latlong"] = (new_df["pickup_latitude"].astype(str)) + ", " + (new_df["pickup_longitude"].astype(str))

non = (new_df["latlong"] == "[") & (new_df["latlong"] == "<")
loc = new_df.drop(index = non.index, inplace = True)
#new_df.drop(index = df["latlong"].index, inplace = True).contains("[")
#loc = str([new_df["latlong"]])
#loc = (str(list))

m = fl.Map(location = [40.693943, -73.985880], control_scale = True, zoom_start = 15)
fl.Marker([40.693943, -73.985880], tooltip = "<b>click for more</b>", popup = "<b>Route 1</b>").add_to(m)
fl.PolyLine([40.693943, -73.985880]).add_to(m)
#m

TypeError: Location should be a sized variable, for example a list or a tuple, instead got 40.693943 of type <class 'float'>.